In [2]:
import pandas as pd
from pymongo import MongoClient
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
import network_model_evaluation

In [3]:
def read_from_db(database_name='wine_quality_database', 
                 collection_name='wine_quality'):
    
    client = MongoClient('mongodb://localhost:27017/')
    db = client[database_name]
    collection = db[collection_name]
    
    data = list(collection.find())
    
    df = pd.DataFrame(data)
    
    return df

In [4]:
df = read_from_db()

In [5]:
df.head()

,_id,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,6674674872fc05f8198b3649,7.4,0.590,0.08,4.4,0.086,6.0,29.0,0.9974,3.38,0.50,9.0,4
1,6674674872fc05f8198b364a,5.7,1.130,0.09,1.5,0.172,7.0,19.0,0.9940,3.50,0.48,9.8,4
2,6674674872fc05f8198b364b,8.8,0.610,0.30,2.8,0.088,17.0,46.0,0.9976,3.26,0.51,9.3,4
3,6674674872fc05f8198b364c,4.6,0.520,0.15,2.1,0.054,8.0,65.0,0.9934,3.90,0.56,13.1,4
4,6674674872fc05f8198b364d,8.3,0.675,0.26,2.1,0.084,11.0,43.0,0.9976,3.31,0.53,9.2,4


In [6]:
df = df.drop(columns=['_id'])

In [7]:
X = df.drop(columns=['quality'])
y = df['quality']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [9]:
scaler = MinMaxScaler()

In [10]:
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [11]:
#label range shift
y_train_shifted = y_train - 4
y_test_shifted = y_test - 4

In [12]:
#labels conversion to one-hot vectors
y_train_one_hot = to_categorical(y_train_shifted)
y_test_one_hot = to_categorical(y_test_shifted)

In [13]:
def create_model():
    #input layer
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(5, activation='relu', input_shape = (X_train_scaled.shape[1], )))
    #hidden layers
    model.add(tf.keras.layers.Dense(4, activation='relu'))
    model.add(tf.keras.layers.Dense(3, activation='relu'))
    #output layer
    model.add(tf.keras.layers.Dense(4, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [14]:
#packaging the model in Sci-Keras
keras_classifier = KerasClassifier(model=create_model, epochs=100, batch_size=10, verbose=1)

In [15]:
#definition of KFold
kf = KFold(n_splits=5 , shuffle=True , random_state=42)

In [16]:
#cross validation
scores = cross_val_score(keras_classifier, X_train_scaled, y_train_one_hot, cv=kf, scoring='accuracy')

Epoch 1/100
48/48 [==============================] - 1s 2ms/step - loss: 1.3849 - accuracy: 0.2589
Epoch 2/100
48/48 [==============================] - 0s 1ms/step - loss: 1.3822 - accuracy: 0.3069
Epoch 3/100
48/48 [==============================] - 0s 1ms/step - loss: 1.3779 - accuracy: 0.3424
Epoch 4/100
48/48 [==============================] - 0s 1ms/step - loss: 1.3715 - accuracy: 0.3194
Epoch 5/100
48/48 [==============================] - 0s 1ms/step - loss: 1.3633 - accuracy: 0.3466
Epoch 6/100
48/48 [==============================] - 0s 1ms/step - loss: 1.3527 - accuracy: 0.3549
Epoch 7/100
48/48 [==============================] - 0s 1ms/step - loss: 1.3402 - accuracy: 0.3633
Epoch 8/100
48/48 [==============================] - 0s 1ms/step - loss: 1.3252 - accuracy: 0.3820
Epoch 9/100
48/48 [==============================] - 0s 1ms/step - loss: 1.3088 - accuracy: 0.3758
Epoch 10/100
48/48 [==============================] - 0s 1ms/step - loss: 1.2933 - accuracy: 0.3820
Epoch 11/

In [17]:
print(f'Mean classification accuracy for neural network model: {scores.mean()}')

Mean classification accuracy for neural network model: 0.4691036414565827
